Aqui vamos criar a dsl, coisas importantes de ter:
- funções de ordem superior (recebendo funções como parâmetro e retornando funções);

- closure;

- macros higiênicas

In [ ]:
(define-macro funcao (parametros)
 (...) )